In [21]:
import os
import subprocess
import numpy as np
import sys
module_path = "/data/luojaa/eukgen/"
sys.path.append(module_path)
from core_functions.helper_functions import fasta_to_dict, dict_to_fasta, filter_by_entropy 
import pandas as pd
# from core_functions.parse_icarus_align import *
# from polvir.parseIcarus import *
from Bio import AlignIO
from Bio.Align import MultipleSeqAlignment
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
import random
from multiprocessing import Pool

In [3]:
alignment_root = "/data/luojaa/polvir/alignments/icarus_cropped/"
alignment_file = "icarus_380.foldmason_aa.entropy_filt.fasta"
# alignment_root = "/data/luojaa/polvir/alignments/uncropped/"
# alignment_file = "poly_380.foldmason_aa.entropy_filt.fasta"
msa_path = f"{alignment_root}{alignment_file}"
outdir = "shuffled"
out_root = f"{alignment_root}{outdir}/"
alignment = AlignIO.read(msa_path, "fasta")

In [5]:
alignment_np = np.array([list(row.seq) for row in alignment])
ncols = alignment_np.shape[1]
n_replicates = 100

for x in range(1,100):
    nresample = int(ncols * x / 100)
    for n in range(n_replicates):
        alignment_shuffled = alignment_np.copy()
        
        # round down
        ind_toreplace = set(random.sample(range(ncols), nresample))
        ind_source = np.random.choice(sorted(set(range(ncols)) - ind_toreplace), size=nresample, replace=True)
        for target, source in zip(ind_toreplace, ind_source):
            alignment_shuffled[:, target] = alignment_np[:, source]
            
        # convert back to MSA
        records = []
        for i in range(alignment_np.shape[0]):
            sequence = ''.join(alignment_shuffled[i])
            
            # Create sequence record
            record = SeqRecord(
                Seq(sequence),
                id=alignment[i].id,
                name=alignment[i].name,
                description=alignment[i].description
            )
            records.append(record)
        outfile = alignment_file.replace(".fasta", f".x{x}_rep{n}.fasta")
        AlignIO.write(MultipleSeqAlignment(records), open(f"{out_root}{outfile}", "w"), "fasta")

In [ ]:
for x in range(1,100):
    nresample = int(ncols * x / 100)
    for n in range(n_replicates):
        outfile = alignment_file.replace(".fasta", f".x{x}_rep{n}.fasta")

In [17]:
shuffle_dir = f"{alignment_root}shuffled/"
x_reps = subprocess.run(f"ls {shuffle_dir}*entropy_filt.x*_rep*.fasta", shell = True, capture_output=True, text=True).stdout.strip().split("\n")


#cmd_iqtree = f"iqtree -s {shuffle_dir}icarus_380.foldmason_aa.entropy_filt.{x_rep_}.fasta -m Q.pfam+F -te {icarus_crop_dir}/icarus_380.foldmason_aa.entropy_filt.fasta.treefile --fast --redo --lmap 10000"

In [20]:
alignment_root

'/data/luojaa/polvir/alignments/icarus_cropped/'

In [26]:
def iqtree_single(shuffle_path):
    shuffle_dir = f"{alignment_root}shuffled/"
    cmd_iqtree = f"iqtree -s {shuffle_path} -m Q.pfam+F -te {alignment_root}/icarus_380.foldmason_aa.entropy_filt.fasta.treefile --fast --redo --lmap 10000"
    subprocess.run(cmd_iqtree, shell = True)#, capture_output=True, text=True).stdout.strip().split("\n")


In [30]:
x_reps[3:5]

['/data/luojaa/polvir/alignments/icarus_cropped/shuffled/icarus_380.foldmason_aa.entropy_filt.x10_rep12.fasta',
 '/data/luojaa/polvir/alignments/icarus_cropped/shuffled/icarus_380.foldmason_aa.entropy_filt.x10_rep13.fasta']

In [39]:
with Pool(processes=8) as pool:
    # Map method
    results = pool.imap_unordered(iqtree_single, x_reps[-10:])

In [40]:
 x_reps[-10:]

['/data/luojaa/polvir/alignments/icarus_cropped/shuffled/icarus_380.muscle3D.entropy_filt.x9_rep91.fasta',
 '/data/luojaa/polvir/alignments/icarus_cropped/shuffled/icarus_380.muscle3D.entropy_filt.x9_rep92.fasta',
 '/data/luojaa/polvir/alignments/icarus_cropped/shuffled/icarus_380.muscle3D.entropy_filt.x9_rep93.fasta',
 '/data/luojaa/polvir/alignments/icarus_cropped/shuffled/icarus_380.muscle3D.entropy_filt.x9_rep94.fasta',
 '/data/luojaa/polvir/alignments/icarus_cropped/shuffled/icarus_380.muscle3D.entropy_filt.x9_rep95.fasta',
 '/data/luojaa/polvir/alignments/icarus_cropped/shuffled/icarus_380.muscle3D.entropy_filt.x9_rep96.fasta',
 '/data/luojaa/polvir/alignments/icarus_cropped/shuffled/icarus_380.muscle3D.entropy_filt.x9_rep97.fasta',
 '/data/luojaa/polvir/alignments/icarus_cropped/shuffled/icarus_380.muscle3D.entropy_filt.x9_rep98.fasta',
 '/data/luojaa/polvir/alignments/icarus_cropped/shuffled/icarus_380.muscle3D.entropy_filt.x9_rep99.fasta',
 '/data/luojaa/polvir/alignments/icar